In [127]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import numpy as numpy

In [129]:
driver = webdriver.Chrome()

names, prices, num_products, ratings_overall, solds = [], [], [], [], []
ratings, reviews, num_products1 = [], [], []

url = 'https://www.tokopedia.com/asus-mobile/etalase/zenfone'
driver.get(url)

# Scroll to load products
for scroll in range(6):
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

products = 1
pro = 1
for x in soup.find_all('div', {'class': 'css-1sn1xa2'}):
    try:
        # Get product name and price
        name = x.find('div', {'class': 'prd_link-product-name css-3um8ox'})
        price = x.find('div', {'class': 'prd_link-product-price css-h66vau'})
        rating_overall = x.find('span', {'class' : 'prd_rating-average-text css-t70v7i'})
        sold = x.find('span', {'class': 'prd_label-integrity css-1sgek4h'})

        names.append(name.get_text() if name else None)
        prices.append(price.get_text() if price else None)
        ratings_overall.append(rating_overall.get_text() if rating_overall else None)
        solds.append(sold.get_text() if sold else None)
        num_products.append(products)
        products += 1
        
        # Navigate to product page
        link = x.find('a')['href']
        
        if link:
            driver.get(link)
            
            # Scroll reviews
            for scroll in range(20):
                driver.execute_script("window.scrollBy(0,300)")
                time.sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # Ratings and reviews
            for z in soup.find_all('div', {'class': "css-1k41fl7"}):
                rating = z.find('div', {'data-testid': 'icnStarRating'})
                review = z.find('span', {'data-testid': 'lblItemUlasan'})
                
                ratings.append(rating.get("aria-label") if rating else None)
                reviews.append(review.get_text() if review else None)
                num_products1.append(pro)


            pro += 1
            
    
    except Exception as e:
        print(f"Error: {e}")
        continue

driver.quit()
print('Scraping complete.')


Scraping complete.


In [133]:
def clean_analyst(data):
    data = data.dropna()
    data['price'] = [number[2:] for number in data.price]
    data['price'] = [number.replace('.', '') for number in data.price]
    data['sold'] = [number.replace('+ terjual', '') for number in data.sold]
    data['sold'] = [number.replace(' terjual', '') for number in data.sold]
    
    for number in data.sold:
        k = ['1rb', '2rb', '3rb', '4rb', '5rb', '6rb', '7rb', '8rb', '9rb', '10rb']
        if number in k:
            data['sold'] = data['sold'].replace(number, number[:1] + '000')
    return data

def clean_review(df):
    df = df.dropna()
    df['rating'] = df['rating'].str.extract(r'bintang (\d+\.?\d*)')
    df['brand'] = ['realme' for i in range(len(df['rating']))]
    df['rating'] = df['rating'].astype(int)
    df['rating_label'] = df['rating'].apply(
        lambda x: 'bad' if x < 3 else ('neutral' if x == 3 else 'good')
    )
    return df

In [ ]:
data['ram'] = data['name'].str.extract(r'(\d+/)')
data['ram'] = data['ram'].fillna(data['name'].str.extract(r'(\d+GB\+)').iloc[:, 0])
data['ram'] = data['ram'].fillna(data['name'].str.extract(r'(\d+\+)').iloc[:, 0])
data['ram'] = data['ram'].fillna(data['name'].str.extract(r'(\d+GB/)').iloc[:, 0])
data['ram'] = data['ram'].fillna('8')
# data['ram'] = [number.replace('256GB', '8') for number in data.ram]
data['ram'] = [number.replace('/', '') for number in data.ram]
data['ram'] = [number.replace('+', '') for number in data.ram]
data['ram'] = [number.replace('GB', '') for number in data.ram]

# data['rom'] = data['name'].str.extract(r'(\+\d+GB)')
data['rom'] = data['name'].str.extract(r'(/\d+GB)')
data['rom'] = data['rom'].fillna(data['name'].str.extract(r'(\+\d+GB)').iloc[:, 0])
data['rom'] = data['rom'].fillna('/32')
data['rom'] = [number[1:] for number in data['rom']]
data['rom'] = [number.replace('GB', '') for number in data.rom]

In [130]:
print(len(names))
print(len(prices))
print(len(num_products))
print(len(ratings_overall))
print(len(solds))

15
15
15
15
15


In [134]:
data = pd.DataFrame({'product_id':num_products,'name':names, 'price':prices, 'rating':ratings_overall, 'sold':solds})
data = clean_analyst(data)
data.head()

,product_id,name,price,rating,sold
0,1,Asus Zenfone 10 8/128GB - Starry Blue,8999000,4.8,60
1,2,Asus Zenfone 10 8/128GB - Aurora Green,8999000,4.9,60
2,3,Asus Zenfone 10 8/128GB - Eclipse Red,8999000,5.0,70
3,4,Asus Zenfone 10 8/128GB - Comet White,8999000,4.8,50
4,5,Asus Zenfone 10 8/128GB - Midnight Black,8999000,4.9,100


In [136]:
print(len(ratings))
print(len(reviews))
print(len(num_products1))

322
322
322


In [ ]:
data1 = pd.DataFrame({'product_id':num_products1, 'rating':ratings, 'review':reviews})
data1 = clean_review(data1)

,product_id,rating,review
0,1,bintang 4,"sesuai pesanan, semoga awet"
1,1,bintang 5,mantap lah utk brg ori dan segel. casing bawaa...
2,1,bintang 5,paking aman dan rapi \ndapat discount pula
3,1,bintang 4,"Dalam hal pengiriman, barangnya sampai dirumah..."
4,1,bintang 5,"penjual oke mantap, gercep bantu ketika ada ke..."


In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  400 non-null    int64 
 1   rating      400 non-null    object
 2   review      384 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.5+ KB
